In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Alagoas - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [9]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_AL.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Alagoas - Produção de Cimento (t)', axis=1)
data

,Ano,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH,...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Alagoas - Consumo de Cimento (t)
0,2008,8.543035,0.601766,2.751323e+07,1.369962e+06,7.684387,2.402206e+07,0.749235,0.622573,0.655354,...,2.868108,11.760983,19.541805,1004.903885,3228.342662,26.070097,20.787810,74.407297,3.079016,446.335000
1,2009,8.571316,0.606396,2.770792e+07,1.372464e+06,7.689682,2.403833e+07,0.751769,0.624024,0.658104,...,2.305885,9.472585,20.736709,1004.528048,1611.970867,25.643630,21.162947,77.670858,2.815837,471.072000
2,2010,8.539868,0.610714,2.842678e+07,1.369925e+06,7.728365,2.421845e+07,0.753767,0.625020,0.660450,...,1.958246,9.332485,19.768388,1004.613084,1481.706116,25.439288,21.037877,77.914366,2.782447,583.772000
3,2011,8.526852,0.614717,2.911152e+07,1.363070e+06,7.768204,2.438475e+07,0.755240,0.625568,0.662401,...,2.079088,10.985689,18.126355,1004.876381,1381.865715,25.362543,20.922266,77.670210,2.776959,742.881000
4,2012,12.945391,0.618592,2.970505e+07,1.349554e+06,7.784639,2.449458e+07,0.756056,0.625434,0.663873,...,2.183282,8.159014,17.184340,1005.022203,1424.662145,25.342566,20.944017,77.821942,2.740248,844.864000
5,2013,13.577910,0.621225,3.023816e+07,1.337956e+06,7.798417,2.459006e+07,0.756644,0.625304,0.664946,...,2.403445,7.913561,17.252127,1005.229473,1469.493466,25.323659,21.009855,78.181963,2.678582,791.375000
6,2014,14.169213,0.625244,3.070999e+07,1.324714e+06,7.820611,2.467763e+07,0.756698,0.624996,0.666358,...,2.748397,10.350442,16.874158,1005.478116,1490.005745,25.397446,21.064696,78.095570,2.657262,818.645143
7,2015,14.659464,0.627431,3.107611e+07,1.310803e+06,7.837546,2.473871e+07,0.756172,0.623329,0.666388,...,2.501529,12.454398,16.443616,1005.522737,1495.198000,25.490999,21.080729,77.734091,2.687007,676.300114
8,2016,14.776623,0.643045,3.127251e+07,1.307005e+06,7.840584,2.474349e+07,0.767870,0.631369,0.678051,...,2.073464,13.111908,16.926315,1005.453192,1487.897624,25.451016,21.102972,78.001380,2.705435,538.586743
9,2017,14.347557,0.643776,3.135879e+07,1.302182e+06,7.848958,2.475540e+07,0.767131,0.630858,0.677874,...,2.177386,9.479812,17.228816,1005.412538,1488.339609,25.438238,21.210017,78.555457,2.646980,362.791000


In [10]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,8.543035,0.601766,2.751323e+07,1.369962e+06,7.684387,2.402206e+07,0.749235,0.622573,0.655354,5.483335e+08,...,-7723.574973,2.868108,11.760983,19.541805,1004.903885,3228.342662,26.070097,20.787810,74.407297,3.079016
1,8.571316,0.606396,2.770792e+07,1.372464e+06,7.689682,2.403833e+07,0.751769,0.624024,0.658104,5.302764e+08,...,-8961.781709,2.305885,9.472585,20.736709,1004.528048,1611.970867,25.643630,21.162947,77.670858,2.815837
2,8.539868,0.610714,2.842678e+07,1.369925e+06,7.728365,2.421845e+07,0.753767,0.625020,0.660450,5.046425e+08,...,-9895.682385,1.958246,9.332485,19.768388,1004.613084,1481.706116,25.439288,21.037877,77.914366,2.782447
3,8.526852,0.614717,2.911152e+07,1.363070e+06,7.768204,2.438475e+07,0.755240,0.625568,0.662401,4.854622e+08,...,-9989.170714,2.079088,10.985689,18.126355,1004.876381,1381.865715,25.362543,20.922266,77.670210,2.776959
4,12.945391,0.618592,2.970505e+07,1.349554e+06,7.784639,2.449458e+07,0.756056,0.625434,0.663873,4.867661e+08,...,-9429.326328,2.183282,8.159014,17.184340,1005.022203,1424.662145,25.342566,20.944017,77.821942,2.740248
5,13.577910,0.621225,3.023816e+07,1.337956e+06,7.798417,2.459006e+07,0.756644,0.625304,0.664946,5.225848e+08,...,-7157.441850,2.403445,7.913561,17.252127,1005.229473,1469.493466,25.323659,21.009855,78.181963,2.678582
6,14.169213,0.625244,3.070999e+07,1.324714e+06,7.820611,2.467763e+07,0.756698,0.624996,0.666358,5.676413e+08,...,-3453.461327,2.748397,10.350442,16.874158,1005.478116,1490.005745,25.397446,21.064696,78.095570,2.657262
7,14.659464,0.627431,3.107611e+07,1.310803e+06,7.837546,2.473871e+07,0.756172,0.623329,0.666388,5.179392e+08,...,-1442.919387,2.501529,12.454398,16.443616,1005.522737,1495.198000,25.490999,21.080729,77.734091,2.687007
8,14.776623,0.643045,3.127251e+07,1.307005e+06,7.840584,2.474349e+07,0.767870,0.631369,0.678051,4.524144e+08,...,-3128.377228,2.073464,13.111908,16.926315,1005.453192,1487.897624,25.451016,21.102972,78.001380,2.705435
9,14.347557,0.643776,3.135879e+07,1.302182e+06,7.848958,2.475540e+07,0.767131,0.630858,0.677874,4.213689e+08,...,-5386.337108,2.177386,9.479812,17.228816,1005.412538,1488.339609,25.438238,21.210017,78.555457,2.646980


In [11]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     471.072000
1     583.772000
2     742.881000
3     844.864000
4     791.375000
5     818.645143
6     676.300114
7     538.586743
8     362.791000
9     340.278000
10    395.359000
11    581.479000
12    576.578000
13    547.622000
14           NaN
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [12]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.394011,-1.562212,-1.771772,1.267308,-1.824617,-1.828417,-1.530739,-1.310765,-1.525249,1.204752,...,-0.569731,2.026352,0.984445,1.186951,-0.595151,3.297433,1.414461,-1.989133,-2.084147,2.701872
1,-1.383296,-1.252696,-1.630985,1.357232,-1.734035,-1.767156,-1.104764,-0.842247,-1.175856,0.820657,...,-0.838980,0.085558,-0.051710,1.903618,-1.755575,-0.042015,0.116552,0.600096,0.287340,0.438290
2,-1.395211,-0.964094,-1.111174,1.266004,-1.072298,-1.088666,-0.769045,-0.520929,-0.877896,0.275392,...,-1.042057,-1.114490,-0.115145,1.322848,-1.493020,-0.311144,-0.505344,-0.263151,0.464287,0.151102
3,-1.400143,-0.696569,-0.616035,1.019651,-0.390788,-0.462241,-0.521550,-0.344017,-0.630022,-0.132596,...,-1.062386,-0.697344,0.633403,0.338007,-0.680070,-0.517416,-0.738908,-1.061109,0.286869,0.103900
4,0.274014,-0.437536,-0.186847,0.533959,-0.109633,-0.048522,-0.384411,-0.387101,-0.443003,-0.104860,...,-0.940648,-0.337667,-0.646476,-0.226984,-0.229833,-0.428998,-0.799707,-0.910979,0.397127,-0.211851
5,0.513671,-0.261567,0.198648,0.117149,0.126049,0.311122,-0.285597,-0.429212,-0.306719,0.657046,...,-0.446625,0.422337,-0.757614,-0.186328,0.410129,-0.336376,-0.857248,-0.456556,0.658738,-0.742231
6,0.737712,0.007045,0.539832,-0.358708,0.505729,0.641003,-0.276579,-0.528629,-0.127318,1.615453,...,0.358807,1.613108,0.345772,-0.413022,1.177832,-0.293997,-0.632685,-0.078042,0.595960,-0.925602
7,0.923465,0.153222,0.804578,-0.858614,0.795419,0.871076,-0.364842,-1.066695,-0.123527,0.558230,...,0.796000,0.760923,1.298413,-0.671248,1.315604,-0.283270,-0.347965,0.032622,0.333289,-0.669771
8,0.967856,1.196863,0.946595,-0.995086,0.847385,0.889099,1.600990,1.528559,1.358195,-0.835563,...,0.429496,-0.716758,1.596124,-0.381740,1.100879,-0.298352,-0.469649,0.186147,0.527516,-0.511267
9,0.805286,1.245745,1.008989,-1.168405,0.990653,0.933950,1.476783,1.363339,1.335691,-1.495939,...,-0.061498,-0.358019,-0.048438,-0.200310,0.975357,-0.297439,-0.508538,0.924979,0.930139,-1.014033


In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     471.072000
1     583.772000
2     742.881000
3     844.864000
4     791.375000
5     818.645143
6     676.300114
7     538.586743
8     362.791000
9     340.278000
10    395.359000
11    581.479000
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [14]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.540735,1.153503,0.54759,-0.669467,0.504915,0.391992,0.441591,0.935129,0.989474,1.108405,...,0.964765,-1.482779,-2.260447,-1.676368,-0.895465,-0.283979,1.831602,0.665102,-1.743991,0.61839


In [15]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    576.578
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [84]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(1, split_factor + 1):
        pos = df.shape[0] - (i*2 + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [112]:
def validation_row_splitter(df): 
    pos = df.shape[0] - 2
    return df.drop(pos), df.iloc[pos:pos + 1]

In [116]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_row_splitter(train_input)
    target,target_val = validation_row_splitter(train_target)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [119]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [120]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1420617233, 2088933431, 1534038316, 414741603, 1143009132, 4066405042, 687189617, 449114294, 255656662, 3167908119, 1898923482, 2336878275, 4293234613, 3728370911, 197783823, 3339352905, 808988921, 1708750558, 4142239872, 3067278310, 894530086, 4246200001, 1769022999, 1070161391, 999394786]


Step: 0 ___________________________________________
val_loss: 10009.798828125
winner_seed: 1420617233


Step: 1 ___________________________________________
val_loss: 605.8226318359375
winner_seed: 2088933431


Step: 2 ___________________________________________
val_loss: 1930.648681640625


Step: 3 ___________________________________________
val_loss: 10010.7080078125


Step: 4 ___________________________________________
val_loss: 640.0399780273438


Step: 5 ___________________________________________
val_loss: 0.0028652288019657135
winner_seed: 4066405042


Step: 6 ___________________________________________
val_loss: 0.24363237619400024


Step: 7 ___________________________________________
val_

In [121]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 487ms/step - loss: 405180.4688 - val_loss: 142854.9062
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 404224.9062 - val_loss: 83962.8828
Epoch 3/10000
1/1 [==============================] - 0s 15ms/step - loss: 451320.1250 - val_loss: 375698.6562
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 303832.5938 - val_loss: 437012.7500
Epoch 5/10000
1/1 [==============================] - 0s 15ms/step - loss: 290580.5625 - val_loss: 261353.5312
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 310496.0000 - val_loss: 555531.0000
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 227898.9375 - val_loss: 427231.2188
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 218173.2500 - val_loss: 532622.3750
Epoch 9/10000
1/1 [==============================] - 0s 15ms/step - loss: 209612.6875 - val_loss: 365952.1250
Epoch 10/1

Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 85737.4062 - val_loss: 125296.8984
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 84215.9531 - val_loss: 124958.0000
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 86942.9688 - val_loss: 105307.6562
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 82592.8906 - val_loss: 119217.9219
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 86396.5469 - val_loss: 140881.3281
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 80540.4609 - val_loss: 114617.3516
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 79161.2656 - val_loss: 118733.4922
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 77993.4766 - val_loss: 107917.2656
Epoch 83/10000
1/1 [==============================] - 0s 14ms/step - loss: 76466.0156 - val_loss: 105949.5312
Epoch 84/1

1/1 [==============================] - 0s 15ms/step - loss: 37161.0117 - val_loss: 50897.1172
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 38890.3477 - val_loss: 44067.6992
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 38859.7969 - val_loss: 44294.1523
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 37645.8711 - val_loss: 42772.7500
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 37080.0781 - val_loss: 42562.2148
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 34336.1719 - val_loss: 48017.6992
Epoch 155/10000
1/1 [==============================] - 0s 14ms/step - loss: 37241.3086 - val_loss: 31580.2129
Epoch 156/10000
1/1 [==============================] - 0s 15ms/step - loss: 38246.6875 - val_loss: 41975.2930
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 33905.1523 - val_loss: 52299.1445
Epoch 158/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 21820.2070 - val_loss: 12937.0908
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 19143.1738 - val_loss: 16088.2070
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 21785.7754 - val_loss: 11564.4863
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 21301.2266 - val_loss: 12179.5020
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 20879.2188 - val_loss: 12303.8447
Epoch 228/10000
1/1 [==============================] - 0s 14ms/step - loss: 21067.3047 - val_loss: 12245.0352
Epoch 229/10000
1/1 [==============================] - 0s 14ms/step - loss: 18790.7441 - val_loss: 17295.2520
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 18946.9805 - val_loss: 13974.3311
Epoch 231/10000
1/1 [==============================] - 0s 14ms/step - loss: 21011.9863 - val_loss: 11588.0615
Epoch 232/10000
1/1 [=====

1/1 [==============================] - 0s 15ms/step - loss: 15561.8242 - val_loss: 3303.5342
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 17936.7871 - val_loss: 8324.3018
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 18085.8516 - val_loss: 3699.2886
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 16366.6660 - val_loss: 2179.5288
Epoch 302/10000
1/1 [==============================] - 0s 14ms/step - loss: 16164.6406 - val_loss: 2245.1426
Epoch 303/10000
1/1 [==============================] - 0s 14ms/step - loss: 16000.9023 - val_loss: 2501.5444
Epoch 304/10000
1/1 [==============================] - 0s 14ms/step - loss: 16319.6211 - val_loss: 2781.9604
Epoch 305/10000
1/1 [==============================] - 0s 13ms/step - loss: 15923.3496 - val_loss: 2159.6396
Epoch 306/10000
1/1 [==============================] - 0s 14ms/step - loss: 15886.0195 - val_loss: 2049.4272
Epoch 307/10000
1/1 [==============

...

1/1 [==============================] - 0s 15ms/step - loss: 13478.6221 - val_loss: 727.8304
Epoch 909/10000
1/1 [==============================] - 0s 14ms/step - loss: 13300.0059 - val_loss: 547.1014
Epoch 910/10000
1/1 [==============================] - 0s 15ms/step - loss: 13264.1055 - val_loss: 528.7446
Epoch 911/10000
1/1 [==============================] - 0s 15ms/step - loss: 13351.4248 - val_loss: 651.5191
Epoch 912/10000
1/1 [==============================] - 0s 15ms/step - loss: 13341.8320 - val_loss: 609.4395
Epoch 913/10000
1/1 [==============================] - 0s 14ms/step - loss: 13271.3379 - val_loss: 558.7868
Epoch 914/10000
1/1 [==============================] - 0s 14ms/step - loss: 13431.2285 - val_loss: 707.1497
Epoch 915/10000
1/1 [==============================] - 0s 14ms/step - loss: 13411.7666 - val_loss: 714.4103
Epoch 916/10000
1/1 [==============================] - 0s 14ms/step - loss: 13268.2275 - val_loss: 564.0031
Epoch 917/10000
1/1 [=======================

1/1 [==============================] - 0s 14ms/step - loss: 11624.4229 - val_loss: 852.9134
Epoch 984/10000
1/1 [==============================] - 0s 15ms/step - loss: 12825.4219 - val_loss: 1361.3911
Epoch 985/10000
1/1 [==============================] - 0s 15ms/step - loss: 12953.5596 - val_loss: 898.9713
Epoch 986/10000
1/1 [==============================] - 0s 14ms/step - loss: 14254.4717 - val_loss: 2302.7991
Epoch 987/10000
1/1 [==============================] - 0s 14ms/step - loss: 13454.6621 - val_loss: 1162.2361
Epoch 988/10000
1/1 [==============================] - 0s 14ms/step - loss: 13513.9941 - val_loss: 1532.8104
Epoch 989/10000
1/1 [==============================] - 0s 14ms/step - loss: 14461.4873 - val_loss: 938.3767
Epoch 990/10000
1/1 [==============================] - 0s 14ms/step - loss: 14002.8877 - val_loss: 1940.9231
Epoch 991/10000
1/1 [==============================] - 0s 15ms/step - loss: 13372.8965 - val_loss: 1194.4398
Epoch 992/10000
1/1 [=================

1/1 [==============================] - 0s 14ms/step - loss: 13566.7217 - val_loss: 398.1625
Epoch 1059/10000
1/1 [==============================] - 0s 15ms/step - loss: 13323.7695 - val_loss: 538.3354
Epoch 1060/10000
1/1 [==============================] - 0s 15ms/step - loss: 11994.3623 - val_loss: 182.4190
Epoch 1061/10000
1/1 [==============================] - 0s 14ms/step - loss: 13533.0869 - val_loss: 148.0045
Epoch 1062/10000
1/1 [==============================] - 0s 14ms/step - loss: 13956.4092 - val_loss: 274.3862
Epoch 1063/10000
1/1 [==============================] - 0s 14ms/step - loss: 11553.8770 - val_loss: 151.7127
Epoch 1064/10000
1/1 [==============================] - 0s 14ms/step - loss: 12352.4473 - val_loss: 804.0170
Epoch 1065/10000
1/1 [==============================] - 0s 14ms/step - loss: 11832.3535 - val_loss: 771.4147
Epoch 1066/10000
1/1 [==============================] - 0s 14ms/step - loss: 11001.8584 - val_loss: 479.5108
Epoch 1067/10000
1/1 [==============

Epoch 1133/10000
1/1 [==============================] - 0s 14ms/step - loss: 13825.9434 - val_loss: 1657.0201
Epoch 1134/10000
1/1 [==============================] - 0s 14ms/step - loss: 13457.1133 - val_loss: 1307.3583
Epoch 1135/10000
1/1 [==============================] - 0s 14ms/step - loss: 13847.2031 - val_loss: 1949.0468
Epoch 1136/10000
1/1 [==============================] - 0s 14ms/step - loss: 13117.3604 - val_loss: 1333.5647
Epoch 1137/10000
1/1 [==============================] - 0s 14ms/step - loss: 13430.5850 - val_loss: 1336.5173
Epoch 1138/10000
1/1 [==============================] - 0s 14ms/step - loss: 13930.5596 - val_loss: 1623.2085
Epoch 1139/10000
1/1 [==============================] - 0s 14ms/step - loss: 13798.9658 - val_loss: 1889.3783
Epoch 1140/10000
1/1 [==============================] - 0s 14ms/step - loss: 13845.6221 - val_loss: 1514.4967
Epoch 1141/10000
1/1 [==============================] - 0s 14ms/step - loss: 13387.5010 - val_loss: 1297.8199
Epoch 1142

1/1 [==============================] - 0s 14ms/step - loss: 13489.1367 - val_loss: 556.4203
Epoch 1208/10000
1/1 [==============================] - 0s 14ms/step - loss: 13486.7070 - val_loss: 531.0642
Epoch 1209/10000
1/1 [==============================] - 0s 14ms/step - loss: 13479.7217 - val_loss: 550.3096
Epoch 1210/10000
1/1 [==============================] - 0s 14ms/step - loss: 13567.7646 - val_loss: 599.1171
Epoch 1211/10000
1/1 [==============================] - 0s 14ms/step - loss: 13473.6758 - val_loss: 538.6683
Epoch 1212/10000
1/1 [==============================] - 0s 14ms/step - loss: 13211.2617 - val_loss: 655.6854
Epoch 1213/10000
1/1 [==============================] - 0s 14ms/step - loss: 13472.5879 - val_loss: 519.2885
Epoch 1214/10000
1/1 [==============================] - 0s 14ms/step - loss: 13567.6221 - val_loss: 556.2043
Epoch 1215/10000
1/1 [==============================] - 0s 14ms/step - loss: 13819.0771 - val_loss: 761.5632
Epoch 1216/10000
1/1 [==============

In [122]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 36ms/step
[576.578](test_target) - [[448.90015]](prediction) = 127.67785351562497


In [123]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [ ]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.592143,-1.238768,-0.940263,-0.691783,-0.837853,-0.796510,-1.270151,-1.293364,-1.255860,1.148205,...,1.277766,1.308949,1.412346,-0.914375,1.380911,1.411042,1.341066,-1.336272,-1.411478,1.406725
1,1.408288,0.028544,-0.444707,1.414104,-0.567735,-0.613762,0.096525,0.151296,0.064802,0.140893,...,-0.114016,-0.190804,-0.643250,1.391499,-0.954679,-0.623537,-0.281746,1.069113,0.629592,-0.577490
2,-0.816144,1.210223,1.384970,-0.722321,1.405588,1.410272,1.173626,1.142068,1.191057,-1.289098,...,-1.163750,-1.118144,-0.769096,-0.477124,-0.426232,-0.787505,-1.059321,0.267160,0.781885,-0.829235


train_target:


,Alagoas - Consumo de Cimento (t)
0,471.072
1,583.772
2,742.881


test_input:


...

,Alagoas - Consumo de Cimento (t)
0,471.072000
1,583.772000
2,742.881000
3,844.864000
4,791.375000
5,818.645143
6,676.300114
7,538.586743
8,362.791000
9,340.278000


test_input:


,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
10,0.752493,1.471361,1.033462,-1.266742,1.053308,0.870414,1.32656,1.501892,1.468511,-1.685451,...,1.657697,-0.43351,-1.873732,-1.10181,0.246107,-0.253551,1.559336,1.807057,-0.460575,-0.291959


test_target:


,Alagoas - Consumo de Cimento (t)
10,395.359


In [ ]:
display(targets)
display(predictions)

In [ ]:
mae = mean_absolute_error(predictions, targets)
mae

In [ ]:
porcentage = mae/np.mean(targets)
porcentage